In [1]:
from dotenv import load_dotenv
# API KEY 정보 불러오기
load_dotenv()

True

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다"),
        ("user", "#Question:\n{question}"),
    ]
)
# 모델
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 출력파서
output_parser = StrOutputParser()

# 체인생성
chain = prompt | llm | output_parser

user_input = "대한민국의 주요 관광지를 알려줘"
response = chain.invoke({"question": user_input})
print(response)

user_input = "위 답변을 요약해줘."
response = chain.invoke({"question": user_input})
print(response)
# 이전 대화를 기억하지 못함

# 대화 맥락을 기억하도록 수정

In [ ]:
#수정
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

# 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다"),
         MessagesPlaceholder(variable_name="chat_history"),  # 대화 맥락 유지
        ("user", "{question}"),
    ]
)
# 모델
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 출력파서
output_parser = StrOutputParser()

# 메모리 객체 생성
memory = ConversationBufferMemory(return_messages=True)

# 체인생성
chain = prompt | llm | output_parser

# 세션별 히스토리 저장소
session_histories = {}

# 세션 히스토리 리턴 함수 (필수!)
def get_session_history(session_id):
    if session_id not in session_histories:
        session_histories[session_id] = ChatMessageHistory()
    return session_histories[session_id]

# 히스토리 기반 체인 생성
chat_chain = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="question",        # 사용자 메시지 키
    history_messages_key="chat_history",  # 프롬프트에서 사용하는 히스토리 키
)

# 세션 ID (하나의 대화 흐름 단위)
session_id = "user-session-001"

In [ ]:
user_input = "대한민국의 주요 관광지를 알려줘"
response = chat_chain.invoke({"question": user_input}, config={"configurable": {"session_id": session_id}})
print(response)

대한민국에는 다양한 매력을 가진 관광지가 많이 있습니다. 다음은 몇 가지 주요 관광지입니다:

1. **서울**
   - **경복궁**: 조선 시대의 대표적인 궁궐로, 한국의 전통 건축을 감상할 수 있습니다.
   - **북촌 한옥마을**: 전통 한옥이 잘 보존된 지역으로, 한국의 전통 가옥을 체험할 수 있습니다.
   - **N서울타워**: 서울의 전경을 한눈에 볼 수 있는 전망대입니다.
   - **명동**: 쇼핑과 다양한 먹거리를 즐길 수 있는 번화가입니다.

2. **부산**
   - **해운대 해수욕장**: 한국에서 가장 유명한 해변 중 하나로, 여름철 많은 관광객이 찾습니다.
   - **감천문화마을**: 알록달록한 집들이 모여 있는 예술적인 마을입니다.
   - **자갈치 시장**: 신선한 해산물을 맛볼 수 있는 전통 시장입니다.

3. **경주**
   - **불국사**: 유네스코 세계문화유산으로 지정된 사찰로, 아름다운 건축물과 자연 경관을 자랑합니다.
   - **석굴암**: 불국사와 함께 유네스코 세계문화유산으로 지정된 석굴 사원입니다.
   - **첨성대**: 동양에서 가장 오래된 천문대 중 하나입니다.

4. **제주도**
   - **한라산**: 제주도의 중심에 위치한 산으로, 등산과 자연 경관을 즐길 수 있습니다.
   - **성산일출봉**: 일출 명소로 유명하며, 유네스코 세계자연유산으로 지정되었습니다.
   - **천지연 폭포**: 아름다운 폭포와 주변 자연 경관이 인상적입니다.

5. **전주**
   - **전주 한옥마을**: 전통 한옥이 밀집해 있는 지역으로, 한국의 전통 문화를 체험할 수 있습니다.
   - **전주비빔밥**: 전주의 대표적인 음식으로, 다양한 재료가 어우러진 맛을 즐길 수 있습니다.

이 외에도 대한민국에는 다양한 자연 경관과 문화유산이 많으니, 여행 계획 시 여러 지역을 탐방해 보시길 추천합니다.


In [ ]:
user_input = "위 답변을 요약해줘."
response = chat_chain.invoke({"question": user_input}, config={"configurable": {"session_id": session_id}})
print(response)

대한민국의 주요 관광지로는 서울의 경복궁, 북촌 한옥마을, N서울타워, 명동, 부산의 해운대 해수욕장, 감천문화마을, 자갈치 시장, 경주의 불국사, 석굴암, 첨성대, 제주도의 한라산, 성산일출봉, 천지연 폭포, 전주의 전주 한옥마을과 전주비빔밥 등이 있습니다. 각 지역은 전통 문화, 자연 경관, 현대적 매력을 다양하게 제공합니다.


In [24]:
# 테스트 입력2
user_input2 = "내 이름은 양경성이야. 반가워"
response = chat_chain.invoke({"question": user_input2}, config={"configurable": {"session_id": session_id}})
print(response)

반가워요, 양경성님! 만나서 정말 기쁩니다. 오늘 어떻게 도와드릴까요?


In [25]:
# 테스트 입력2
user_input2 = "내 이름이 뭐라고?"
response = chat_chain.invoke({"question": user_input2}, config={"configurable": {"session_id": session_id}})
print(response)

당신의 이름은 양경성이에요. 맞나요?
